In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random, gc, keras

from keras import backend as K
from keras.utils import plot_model
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.applications.vgg16 import VGG16

%matplotlib inline

Using TensorFlow backend.


# Define Parameters

In [2]:
img_size = (150, 150, 3)  # target image size
margin = 0.3              # triplet loss margin
batch_size = 64           # training batch size

In [3]:
# Implement pre-trained VGG16 CNN model
vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=img_size)
plot_model(vgg16, show_shapes=True, to_file='./result/vgg16.pdf')
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

# Define Triplet Network

In [4]:
# Define base network for triplet network
def base_net(input_shape=(150, 150, 3), trainable=False):
    """ define triplet network """
    # load pre-trained VGG16 model
    vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    vgg16.trainable = trainable
    
    # define sequential model
    model = Sequential(name='base_net')
    model.add(vgg16)
    model.add(Flatten(name='flatten'))
    model.add(Dense(512, activation='relu', name='fc1'))
    model.add(Dense(128, activation=None, name='fc2'))
    model.add(Lambda(lambda x: K.l2_normalize(x, axis=1), name='l2_norm'))
    
    return model

In [5]:
base_model = base_net(input_shape=(150, 150, 3), trainable=False)
plot_model(base_model, show_shapes=True, to_file='./result/base_model.pdf')
base_model.summary()

Instructions for updating:
dim is deprecated, use axis instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 512)               4194816   
_________________________________________________________________
fc2 (Dense)                  (None, 128)               65664     
_________________________________________________________________
l2_norm (Lambda)             (None, 128)               0         
Total params: 18,975,168
Trainable params: 4,260,480
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
# Define triplet network
def triplet_net(base_model, input_shape=(150, 150, 3)):
    """ function to define triplet networks """
    # define input: anchor, positive, negative
    anchor = Input(shape=input_shape, name='anchor_input')
    positive = Input(shape=input_shape, name='positive_input')
    negative = Input(shape=input_shape, name='negative_input')
    
    # extract vector represent using CNN based model
    anchor_vec = base_model(anchor)
    pos_vec = base_model(positive)
    neg_vec = base_model(negative)
    
    # define inputs and outputs
    inputs=[anchor, positive, negative]
    outputs=[anchor_vec, pos_vec, neg_vec]
    
    # define the triplet model
    model = Model(inputs=inputs, outputs=outputs, name='triplet_net')
    
    return model

In [7]:
triplet_model = triplet_net(base_model=base_model, input_shape=(150, 150, 3))
plot_model(triplet_model, show_shapes=True, to_file='./result/triplet_network.pdf')
triplet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
base_net (Sequential)           (None, 128)          18975168    anchor_input[0][0]               
                                                                 positive_input[0][0]             
          